# Project Scenario:
An international firm that is looking to expand its business in different countries across the world has recruited you. You have been hired as a junior Data Engineer and are tasked with creating an automated script that can `extract the list of all countries in order of their GDPs in billion USDs (rounded to 2 decimal places), as logged by the International Monetary Fund (IMF).` Since IMF releases this evaluation twice a year, this code will be used by the organization to extract the information as it is updated.<br>

The required data seems to be available on the URL mentioned below:<br>

URL 'https://web.archive.org/web/20230902185326/https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29'<br>

The required information needs to be made accessible as a `CSV` file `Countries_by_GDP.csv` as well as a table `Countries_by_GDP` in a database file `World_Economies`.db with attributes `Country` and `GDP_USD_billion`.<br>

Your boss wants you to demonstrate the success of this code by running a query on the database table to display `only the entries with more than a 100 billion USD economy`. Also, you should log in a file with the entire process of execution named `etl_project_log.txt`.<br>

You must create a Python code `etl_project_gdp.py` that performs all the required tasks.<br>

In [1]:
# Standard Libraries
import sqlite3
from datetime import datetime # import this for the time stamp fuction
import os

# third party
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

## Declare the Attributes

In [2]:
url = 'https://web.archive.org/web/20230902185326/https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29'
table_attribs = ['Country', 'GDP_USD_millions']
db_name = 'World_Economies.db'
table_name = 'Countries_by_GDP'
csv_path = os.path.join('Countries_by_GDP.csv')

## Extracting information using the web scraping process
Inspect the URL and note the position of the table. Note that even the images with captions in them are stored in tabular format. Hence, in the given webpage, our table is at the third position, or index 2. Among this, we require the entries under 'Country/Territory' and 'IMF -> Estimate'.<br>

Note that there are a few entries in which the `IMF estimate` is shown to be `—`. Also, there is an entry at the top named `World`, which we do not require. Segregate this entry from the others because this entry does not have a hyperlink and all others in the table do. So you can take advantage of that and access only the rows for which the entry under `Country/Terriroty` has a hyperlink associated with it.<br>

Note that '—' is a special character and not a general hyphen, '-'. Copy the character from the instructions here to use in the code.<br>


In [7]:
def extract(url, table_attribs):
    ''' This function extracts the required
    information from the website and saves it to a dataframe. The
    function returns the dataframe for further processing. '''
    
    page = requests.get(url).text # Extract the web page as text
    data = BeautifulSoup(page, 'html.parser') # Parse the text into an HTML object.
    df = pd.DataFrame(columns=table_attribs) # Create an empty pandas DataFrame named df with columns as the table_attribs.
    tables = data.find_all('tbody') # Extract all 'tbody' attributes of the HTML object 
    rows = tables[2].find_all('tr') # and then extract all the rows of the index 2 table using the 'tr' attribute.

    # Check the contents of each row, having attribute ‘td’, for the following conditions.
    # a. The row should not be empty.
    # b. The first column should contain a hyperlink.
    # c. The third column should not be '—'.
    for row in rows:
        col = row.find_all('td') 
        if len(col)!=0:
            if col[0].find('a') is not None and '—' not in col[2]:
                data_dict = {"Country": col[0].a.contents[0],
                            "GDP_USD_millions": col[2].contents[0]}
                df1 = pd.DataFrame(data_dict, index=[0])
                df = pd.concat([df,df1], ignore_index=True)
    return df

df = extract(url, table_attribs)
df


,Country,GDP_USD_millions
0,United States,"26,854,599"
1,China,"19,373,586"
2,Japan,"4,409,738"
3,Germany,"4,308,854"
4,India,"3,736,882"
...,...,...
186,Marshall Islands,291
187,Palau,262
188,Kiribati,248
189,Nauru,151


# Transform the Info
The transform function needs to modify the `GDP_USD_millions`. 

In [8]:
def transform(df):
    ''' This function converts the GDP information from Currency
    format to float value, transforms the information of GDP from
    USD (Millions) to USD (Billions) rounding to 2 decimal places.
    The function returns the transformed dataframe.'''

    # Convert the contents of the 'GDP_USD_millions' column of df dataframe from currency format to floating numbers.
    GDP_list = df["GDP_USD_millions"].tolist()
    GDP_list = [float("".join(x.split(','))) for x in GDP_list]
    GDP_list = [np.round(x/1000,2) for x in GDP_list]
    df["GDP_USD_millions"] = GDP_list
    df=df.rename(columns = {"GDP_USD_millions":"GDP_USD_billions"})
    return df

transformed_df = transform(df)
transformed_df

,Country,GDP_USD_billions
0,United States,26854.60
1,China,19373.59
2,Japan,4409.74
3,Germany,4308.85
4,India,3736.88
...,...,...
186,Marshall Islands,0.29
187,Palau,0.26
188,Kiribati,0.25
189,Nauru,0.15


## Loading information to a CSV file and a database

In [30]:
def load_to_csv(df, csv_path):
    ''' This function saves the final dataframe as a `CSV` file 
    in the provided path. Function returns nothing.'''
    df.to_csv(csv_path)

# load_to_csv(transformed_df,csv_path)

In [34]:
def load_to_db(df, sql_connection, table_name):
    ''' This function saves the final dataframe as a database table
    with the provided name. Function returns nothing.'''
    df.to_sql(table_name, sql_connection, if_exists='replace', index=False)

## Querying the database table

In [33]:
def run_query(query_statement, sql_connection):
    ''' This function runs the stated query on the database table and
    prints the output on the terminal. Function returns nothing. '''
    query_output = pd.read_sql(query_statement, sql_connection)
    print(query_output)

## Logging progress
This function will be called multiple times throughout the execution of this code and will be asked to add a log entry in a .txt file, `etl_project_log.txt`. The entry is supposed to be in the following format:`<Time_stamp> : <message_text>`

In [32]:
def log_progress(message): 
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second 
    now = datetime.now() # get current timestamp 
    timestamp = now.strftime(timestamp_format) 
    with open("./etl_project_log.txt","a") as f: 
        f.write(timestamp + ' : ' + message + '\n')

## Final Function Call

In [35]:
log_progress('Preliminaries complete. Initiating ETL process')
df = extract(url, table_attribs)

log_progress('Data extraction complete. Initiating Transformation process')
df = transform(df)

log_progress('Data transformation complete. Initiating loading process')
load_to_csv(df, csv_path)
log_progress('Data saved to CSV file')

sql_connection = sqlite3.connect('World_Economies.db')
log_progress('SQL Connection initiated.')
load_to_db(df, sql_connection, table_name)
log_progress('Data loaded to Database as table. Running the query')

query_statement = f"SELECT * from {table_name} WHERE GDP_USD_billions >= 100"
run_query(query_statement, sql_connection)
log_progress('Process Complete.')
sql_connection.close()

          Country  GDP_USD_billions
0   United States          26854.60
1           China          19373.59
2           Japan           4409.74
3         Germany           4308.85
4           India           3736.88
..            ...               ...
64          Kenya            118.13
65         Angola            117.88
66           Oman            104.90
67      Guatemala            102.31
68       Bulgaria            100.64

[69 rows x 2 columns]
